In [ ]:
# в этом ноутбуке я считаю профиты для лонг трейдов со скользящим стоплоссом.
# собираюсь использовать это как раметку данных.
# todo: если посмотреть начало каждого участка, где лонг профит становится положительным, то можно заметить, что в 
# начале есть участок снижения цены. цена снижается, но размер снижения не выбивает скользящий стоп лосс. иногда 
# этот участок выглядит как ошибочно помеченным зеленым. видно просто по динамике, что до минимума динамика одна, а 
# после минимума меняется. то, что видно на глаз бывает затруднительно переложить в алгоритм без использования
# нейронок. однако, в данном случае видно, что во время снижения цены, даже не смотря на то, что присутствуют колебания 
# туда-сюда, не снижается красная линия. не снижается она ровно до тех пор, пока не наступит минимум. надо включить 
# блять мозги и понять почему так происходит. я тупой. не понимаю. понятно только, что это можно использовать для того, 
# чтобы улучшить качество разметки и выбросить эти начальные участки, которые ошибочно размечаются как зеленые.
import pandas as pd
import numpy as np
import pandas_ta as ta
import forecast
from quote_chart import create_chart_app
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime
from tsl import get_trailing_stop_loss_long_profits, get_trailing_stop_loss_short_profits

fee_percent = 0.07

In [ ]:
data = forecast.get_df('BTCUSD', '', '1T')

In [ ]:
# xx0 = datetime.strptime('2024-05-20 23:00:00.000000', '%Y-%m-%d %H:%M:%S.%f')
# xx1 = datetime.strptime('2024-05-20 23:30:00.000000', '%Y-%m-%d %H:%M:%S.%f')
# data = data[xx0:xx1]
(long_profits, long_stop_loss_prices, maximums) = get_trailing_stop_loss_long_profits(data, 0.5 * 0.01)
(short_profits, short_stop_loss_prices, minimums) = get_trailing_stop_loss_short_profits(data, 0.5 * 0.01)
# fill None values with last not-None value to avoid issues on the chart.
long_profits_df = pd.DataFrame(long_profits)
long_profits_df.ffill(inplace=True)
#long_profits_df[long_profits_df < 0] = 0
data['long_profit'] = long_profits_df.to_numpy()
short_profits_df = pd.DataFrame(short_profits)
short_profits_df.ffill(inplace=True)
#short_profits_df[short_profits_df < 0] = 0
data['short_profit'] = short_profits_df.to_numpy()
# sl_df = pd.DataFrame(stop_loss_prices)
# sl_df.ffill(inplace=True)
# data['stop_loss_price'] = sl_df.to_numpy()
# maximums_df = pd.DataFrame(maximums)
# maximums_df.ffill(inplace=True)
# data['maximum'] = maximums_df.to_numpy()

In [ ]:
rsi = ta.rsi(data['close'], 94)
data['rsi'] = rsi
rsi10 = ta.ema(rsi, 40)
data['rsi10'] = rsi10
rsi90 = ta.ema(rsi, 90)
data['rsi90'] = rsi90

In [ ]:
resampled_df = data.copy()

# called when a period button is pressed under the plot.
def on_period_change(button_id):
    global resampled_df, selected_period
    if button_id == '':
        return
    selected_period = button_id
    resampled_df = data.resample(selected_period).agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum',
        'long_profit': 'first',
        'short_profit': 'first',
        'rsi': 'first',
        'rsi10': 'first',
        'rsi90': 'first',
    })
    resampled_df = resampled_df.dropna()

def create_figure(x0, x1):
    if x0 is not None:
        #print(x0)
        df = resampled_df[x0:x1]
    else:
        # df = resampled_df[:100]#
        xx0 = datetime.strptime('2024-05-27 00:00:00.000000', '%Y-%m-%d %H:%M:%S.%f')
        xx1 = datetime.strptime('2024-05-28 00:00:00.000000', '%Y-%m-%d %H:%M:%S.%f')
        df = resampled_df[xx0:xx1]
        
    # define multiple panes. The top pane will be for the main price chart with candles. The second pane is for volumes.
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                vertical_spacing=0.01,
                row_heights=[0.8, 0.2],
                specs=[[{"secondary_y": True}], [{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=df.index, y=df['open'] - df['short_profit'], mode='lines', line=dict(color='red'),
                             name='short_profit'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['open'] + df['long_profit'], mode='lines', line=dict(color='green'),
                             name='long_profit'), row=1, col=1)
    fig.add_trace(go.Candlestick(x=df.index,
                                 open=df['open'],
                                 high=df['high'],
                                 low=df['low'],
                                 close=df['close'],
                                 name='ohlc'), row=1, col=1)
    # plot the main chart with price candles.
    # fig.add_trace(go.Scatter(x=df.index, y=df['open'] + df['profit'] - (df['maximum'] - df['close']), 
    #                          mode='lines', line=dict(color='orange'), name='discounted profit'), row=1, col=1)
    # fig.add_trace(go.Scatter(x=df.index, y=df['maximum'], mode='lines', line=dict(color='green'), name='maximum'),
    #               row=1, col=1)

    # fig.add_trace(go.Scatter(x=df.index, y=df['open'] * (1 + 0.01*fee_percent*2), mode='lines', line=dict(color='blue'),
    # name='fee'), row=1, col=1)
    # fig.add_trace(go.Scatter(x=df.index, y=df['stop_loss_price'], mode='lines', line=dict(color='red'),
    #                          name='stop_loss_pirce'), row=1, col=1)
    #fig.add_trace(go.Bar(x=df.index, y=df['volume'], name='Volume'), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['rsi'], name='rsi', line=dict(color='red')), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['rsi10'], name='rsi10', line=dict(color='blue')), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['rsi90'], name='rsi90', line=dict(color='black')), row=2, col=1)
    # fig.add_trace(go.Scatter(x=df.index, 
    #                          y=df['open'] - df['profit'] + df['open'] * 0.01*fee_percent*2,
    #                          mode='lines', line=dict(color='blue'), name='prof wo fee'), row=1, col=1)
    # set the default dragmode to pan, remove the range slider because i use zoom/pan instead of it.
    fig.update_layout(
        dragmode='pan',
        xaxis_rangeslider_visible=False,
        width=1200, # px
        height=600,
        margin=dict(l=50, r=0, t=10, b=0),
        yaxis=dict(side='right'),
        yaxis2=dict(side='left'),
        yaxis3=dict(side='right'),
        yaxis4=dict(side='left'),
        yaxis5=dict(side='right'),
        yaxis6=dict(side='left'),
        )
    fig.update_xaxes(
        ticklabelposition="outside right",  # keep labels on the right so that they don't affect margin-left.
    )
    return fig

app = create_chart_app(create_figure, on_period_change, period_buttons=None, debug=False)
app.run_server(debug=True)

In [ ]:
n=int(data.shape[0]/1100)
n

In [ ]:
shift = 1000
n=10000

In [ ]:
fig = make_subplots(rows=1, cols=1, shared_xaxes=True, 
            vertical_spacing=0.01,
            row_heights=[1],
            specs=[[{"secondary_y": True}]])
rsi_diffs = (data['rsi90'][shift:shift+n] - data['rsi10'][shift:shift+n])
fig.add_trace(go.Scatter(x=rsi_diffs,
                         y=data['short_profit'][shift:shift+n],
                         mode='markers',
                         marker=dict(color='red', size=1)), row=1, col=1)
fig.add_trace(go.Scatter(x=rsi_diffs,
                         y=data['long_profit'][shift:shift+n],
                         mode='markers',
                         marker=dict(color='blue', size=1)), row=1, col=1)